# Reweighting execution pipeline example
### Wolf Gautier
This notebook contains a start-to-finish sample pipeline for working with the ML4EFT framework in reweighted form. It assumes that all dependencies (including LHAPDF) are installed properly, and that you are working with one or more Les Houches event files (.lhe).

## 1. Extracting observables and weights
Having obtained a sample simulated .lhe event file, the proper observables first have to be constructed from the raw data, and the weights should be extracted alongside them. This is done with the pylhe library, as demonstrated in the code below that makes use (original taken from J. ter Hoeve, reworked). Here, the invariant mass $m_{tt}$ is extracted from simulated $t, \bar{t}$-collisions, alongside the weights for two points in the parameter space of some Wilson coefficient.

In [ ]:
import pandas as pd
import pylhe
import ml4eft.preproc.lhe_reader as lhe

path_to_lhe_file = "<your path here>"

# read .lhe
lhe_init = pylhe.read_lhe_init(path_to_lhe_file)

# generate empty lists for storing event values and weights
events = list()
weight_values_sm = list()
weight_values_eft = list()

for e in pylhe.read_lhe_with_attributes(path_to_lhe):

	# create particle instances
	for part in e.particles:
		if part.id == 6: # t
			t = lhe.Kinematics(part)

		elif part.id == -6: #tbar
			tbar = lhe.Kinematics(part)

	# create particle systems
	tt = t + tbar

	# append kinematics, weights
	events.append([tt.get_inv_mass() * 1e-3]) # normalised
	weight_values_sm.append(e.eventinfo.weight)
	weight_values_eft.append([e.weights['<weight 1>'], 
								e.weights['<weight 2']])

# generate pandas dataframes	
df_events = pd.DataFrame(events, columns=['m_tt'])
df_weights_sm = pd.DataFrame(weight_values_sm, columns=['weight'])
df_weights_eft = pd.DataFrame(weight_values_eft, columns=['<w1>', '<w2>'])

# save events
df_events.to_pickle('<your location>', compression='gzip')
df_weights_sm.to_pickle('<your location>', compression='gzip')
df_weights_eft.to_pickle('<your location>', compression='gzip')
					   

## 2. Training a model
Now that the training data and weights are collected in the correct format, they can be fed to a model with the correct settings (see [ML4EFT documentation site](https://lhcfitnikhef.github.io/ML4EFT)). Assuming you have a .json file with the settings you desire to train with, you can now initialise a model and train it.

In [ ]:
import ml4eft.core.classifier as classifier

# coefficients to train in sequence
coeffs = ['ctGRe', 'cHu']
# path to .json containing settings
runcard = "<your path>"
# path where models will be saved
output_dir = "<your path>"

# train model for all provided coefficients
for coefficient in coeffs:
		fitter = classifier.Fitter(
			json_path = runcard,
			mc_run = 1,
			c_name = coefficient, 
			output_dir = output_dir,
			print_log=True
		)

## 3. Sampling output
Now that a trained model is present in the specified output directory, its output can be sampled and compared to benchmark true values by initialising it again.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from ml4eft.core.analyse import analyse

# sample observable values for plotting
x = np.linspace(1.45, 3, 100).reshape(-1, 1)
df_features = pd.DataFrame(x, columns=['m_tt'])

# obtain truth values for comparison
r1_truth = analyse.Analyse.likelihood_ratio_truth(df_features,
													{'c_val_1': -2, 'c_val_2': 2},
													process='tt',
												   	order='lin')
NN1_truth = (r1_truth - 1) / (-2)

# initialise trained model with right architecture
architecture = [1, 10, 10, 10, 1] # layer 1 = 1 node, layer 2 = 10...
network_path = "<your path>"
loaded_model = classifier.Classifier(architecture)
loaded_model.load_state_dict(torch.load(network_path))

# scale features to match those used in training
path_to_scaler = "<your path>" # in model directory
scaler = joblib.load(path_to_scaler)
features_scaled = scaler.transform(df_features)
features_as_tensor = torch.tensor(features_scaled)

# obtain model outputs
with torch.no_grad():
	# linear contributions
	NN_lin = loaded_model.n_alpha_1(features_as_tensor)
	# quadratic contribution
	NN_quad = loaded_model.n_alpha_2(features_as_tensor)

# plot comparison
plt.plot(x, NN_lin, label='linear')
plt.plot(x, NN_quad, label='quadratic')
plt.plot(x, NN1_truth, label='Truth (ctG)', linestyle='dotted')

# plot settings
plt.xlabel('Invariant Mass of t bar [TeV]')
plt.legend()
plt.savefig("<your path>")

Congratulations! You have now performed an unbinned fit of Wilson coefficients of your choosing.